In [1]:
import numpy as np 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import pandas as pd 
import random
import math
import time
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime
import operator
plt.style.use('seaborn')
%matplotlib inline

In [2]:
df = pd.read_csv('C:\\Users\\\Home\\Desktop\\DataTrained\\covid_dataset.csv')


In [3]:
df.head()

,Date,Country,Confirmed,Recovered,Deaths
0,1/22/2020,Afghanistan,0,0,0
1,1/22/2020,Albania,0,0,0
2,1/22/2020,Algeria,0,0,0
3,1/22/2020,Andorra,0,0,0
4,1/22/2020,Angola,0,0,0


In [12]:
def calculate_increase_rate(csv_file: str = "C:\\Users\\\Home\\Desktop\\DataTrained\\covid_dataset.csv"):
    with open(csv_file) as read_file:
        rows = read_file.read().splitlines()
    with open(csv_file, "w") as write_file:
        header_row = rows[0] + ",Increase rate"
        write_file.write(f"{header_row}\n")
        previous_row = None
        for row in rows[1:]:  # exclude header row
            if previous_row:
                prev_confirmed = int(previous_row.split(",")[1])
                confirmed = int(row.split(",")[1])
                rate = (confirmed - prev_confirmed) / prev_confirmed * 100
                write_file.write(f"{row},{rate}\n")
            else:
                write_file.write(f"{row},\n")
            previous_row = row


In [13]:
def adjust_date(s):
    l = s.split("/")
    return f"20{l[2]}-{int(l[0]):02d}-{int(l[1]):02d}"

In [17]:
base_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/"
confirmed_url = "time_series_covid19_confirmed_global.csv"
dead_url = "time_series_covid19_deaths_global.csv"
recovered_url = "time_series_covid19_recovered_global.csv"


In [18]:
print("===============\nWorking on basic time series\n")


Working on basic time series



In [19]:
df = pd.read_csv(base_url + confirmed_url)
confirmed_copy = df.copy()  # for time series file with provinces
df = df.drop(["Lat", "Long"], axis=1)

In [20]:
to_concat = df[df["Province/State"].notna()]["Country/Region"].unique()
# We have to combine the numbers from the countries that are split up into
# provinces

In [21]:
for country in to_concat:
    new_row = df[df["Country/Region"] == country].sum()
    new_row["Country/Region"] = country
    new_row["Province/State"] = np.NaN
    df = df.drop(df[df["Country/Region"] == country].index)
    df = df.append(new_row, ignore_index=True)

In [27]:
confirmed = df[df["Province/State"].isna()].drop(
    "Province/State", axis=1
)  # take only countries (no territories)
# print(to_concate)

In [28]:
df = pd.read_csv(base_url + dead_url)
dead_copy = df.copy()  # for time series file with provinces
df = df.drop(["Lat", "Long"], axis=1)
to_concat = df[df["Province/State"].notna()]["Country/Region"].unique()
for country in to_concat:
    new_row = df[df["Country/Region"] == country].sum()
    new_row["Country/Region"] = country
    new_row["Province/State"] = np.NaN
    df = df.drop(df[df["Country/Region"] == country].index)
    df = df.append(new_row, ignore_index=True)

In [29]:
dead = df[df["Province/State"].isna()].drop(
    "Province/State", axis=1
)  # take only countries (no territories)

In [30]:
# make sure all values in the column are strings (np.NaN is a float)
confirmed["Country/Region"] = confirmed["Country/Region"].fillna("")
dead["Country/Region"] = dead["Country/Region"].fillna("")

In [31]:
df = pd.read_csv(base_url + recovered_url)
recovered_copy = df.copy()  # for time series
df = df.drop(["Lat", "Long"], axis=1)
to_concat = df[df["Province/State"].notna()]["Country/Region"].unique()
for country in to_concat:
    new_row = df[df["Country/Region"] == country].sum()
    new_row["Country/Region"] = country
    new_row["Province/State"] = np.NaN
    df = df.drop(df[df["Country/Region"] == country].index)
    df = df.append(new_row, ignore_index=True)

In [32]:
recovered = df[df["Province/State"].isna()].drop(
    "Province/State", axis=1
)  # take only countries (no territories)
